In [1]:
import random
from icecream import ic
import numpy as np
from tqdm import tqdm


In [2]:
def random_solution(n):
    """Create a random permutation of cities [0..n-1]."""
    perm = list(range(n))
    random.shuffle(perm)
    return perm


def route_length(solution, data):
    """Compute the total distance of the tour given by 'solution'."""
    n = len(solution)
    total = 0.0
    #if a-b-c-d-a ,range(n); else if a-b-c-d , range(n-1)
    #b = solution[(i + 1) % n]; b = solution[i + 1]
    for i in range(n - 1):
        a = solution[i]
        b = solution[i + 1]
        total += float(data[a][b])
    return total

def fitness_order(solutions, data):
    """Sort solutions by tour length ascending (shorter is better)."""
    scored = [(s, route_length(s, data)) for s in solutions]
    scored.sort(key=lambda x: x[1])
    return [s for s, _ in scored]

def ox_crossover(parent1, parent2):
    """Using Standard OX crossover to create a child solution from two parents."""
    n = len(parent1)
    a, b = sorted(random.sample(range(n), 2))

    child = [None] * n
    child[a:b+1] = parent1[a:b+1]

    used = set(child[a:b+1])
    write = (b + 1) % n
    for g in parent2:
        if g not in used:
            child[write] = g
            write = (write + 1) % n

    return child

def swap(solution):
    """Perform swap mutation on perm solution"""
    n = len(solution)
    a, b = random.sample(range(n), 2)
    sol = solution[:]
    sol[a], sol[b] = sol[b], sol[a]
    return sol

def roulette(population, data):
    """Fitness-proportionate selection (shorter length -> higher weight).
    
    For example, [10, 12, 13], max_len = 13, scores = [3, 1, 0],so probability of selecting each is [3/4, 1/4, 0].
    """
    lengths = [route_length(s, data) for s in population]
    mx, mn = max(lengths), min(lengths)

    if mx == mn:
        # All have the same length, select randomly
        return random.choice(population)

    scores = [mx - L for L in lengths]  # > 0
    total = sum(scores)
    r, acc = random.random() , 0.0
    for s, w in zip(population, scores):
        acc += w
        if acc >= r:
            return s
    return population[-1]

In [3]:

def solve(data, populationSize, maxSteps, mutationProbability=0.2):
    """Run GA for a single problem with OX crossover and swap mutation."""
    n = data.shape[0]
    population = [random_solution(n) for _ in range(populationSize)]

    population = [random_solution(n) for _ in range(populationSize)]
    lengths = [route_length(s, data) for s in population]

    best_distance = min(lengths)
    best_solution = population[lengths.index(best_distance)][:]

    history = [best_distance]
    patience = 200        
    restart_frac = 0.30   
    no_improve = 0

    for _ in tqdm(range(maxSteps)):
        events = max(1, populationSize // 6)
        for _e in range(events):

            p1 = roulette(population, data)
            p2 = roulette(population, data)

            child = ox_crossover(p1, p2)
            if random.random() < mutationProbability:
                child = swap(child)
            Lc = route_length(child, data)

            worst_idx = max(range(populationSize), key=lambda i: lengths[i])
            if Lc < lengths[worst_idx]:
                population[worst_idx] = child
                lengths[worst_idx] = Lc

        # update best
        curr_idx = min(range(populationSize), key=lambda i: lengths[i])
        curr_best = lengths[curr_idx]
        if curr_best < best_distance:
            best_distance = curr_best
            best_solution = population[curr_idx][:]
            no_improve = 0
        else:
            no_improve += 1
        history.append(best_distance)

        # restart if no improvement and keep elites
        if no_improve >= patience:
            inject = max(1, int(populationSize * restart_frac))
            worst_indices = sorted(range(populationSize), key=lambda i: lengths[i], reverse=True)[:inject]
            for idx in worst_indices:
                population[idx] = random_solution(n)
                lengths[idx] = route_length(population[idx], data)
            curr_idx = min(range(populationSize), key=lambda i: lengths[i])
            best_distance = lengths[curr_idx]
            best_solution = population[curr_idx][:]
            no_improve = 0
            history.append(best_distance)

    return best_distance, best_solution, history

In [4]:
# Check data directory and load .npy files
DATA_DIR = './lab2'

import os
if not os.path.isdir(DATA_DIR):
    raise SystemExit(f"Data directory not found: {DATA_DIR}")

files = [f for f in os.listdir(DATA_DIR) if f.endswith('.npy')]
files.sort()
if not files:
    raise SystemExit("No .npy files found under ./lab2")

In [5]:
DEFAULT_POP = 10 
DEFAULT_STEPS = 10000 
DEFAULT_MUT = 0.2

all_results = []

In [6]:
for fname in files:
    path = os.path.join(DATA_DIR, fname)
    data = np.load(path, allow_pickle=False)
    n = data.shape[0]

    pop = DEFAULT_POP
    steps = DEFAULT_STEPS
    mut = DEFAULT_MUT

    ic(f"Running {fname} -> N={n}, pop={pop}, steps={steps}, mut={mut}")
    bestDistance, bestSolution, history = solve(
        data=data,
        populationSize=pop,
        maxSteps=steps,
        mutationProbability=mut,
    )

    print("\n== Result ==")
    print(f"file: {fname}")
    print(f"bestDistance: {bestDistance}")
    print(f"bestSolution: {bestSolution}")
    all_results.append((fname, bestDistance, bestSolution))

print("\n=== SUMMARY (FULL) ===")
for name, dist_val, sol in all_results:
    print(f"file: {name}")
    print(f"bestDistance: {dist_val}")
    print(f"bestSolution: {sol}")
    print("-")

ic| f"Running {fname} -> N={n}, pop={pop}, steps={steps}, mut={mut}": 'Running problem_g_10.npy -> N=10, pop=10, steps=10000, mut=0.2'
100%|██████████| 10000/10000 [00:00<00:00, 13334.71it/s]
ic| f"Running {fname} -> N={n}, pop={pop}, steps={steps}, mut={mut}": 'Running problem_g_100.npy -> N=100, pop=10, steps=10000, mut=0.2'



== Result ==
file: problem_g_10.npy
bestDistance: 1243.5632731014168
bestSolution: [1, 6, 4, 5, 9, 7, 0, 8, 3, 2]


100%|██████████| 10000/10000 [00:06<00:00, 1531.98it/s]
ic| f"Running {fname} -> N={n}, pop={pop}, steps={steps}, mut={mut}": 'Running problem_g_1000.npy -> N=1000, pop=10, steps=10000, mut=0.2'



== Result ==
file: problem_g_100.npy
bestDistance: 9394.191746759203
bestSolution: [73, 70, 56, 18, 79, 35, 62, 23, 61, 59, 39, 82, 29, 9, 15, 83, 52, 32, 97, 72, 37, 33, 7, 27, 99, 71, 63, 47, 74, 20, 14, 41, 0, 48, 31, 12, 40, 43, 88, 92, 86, 4, 5, 22, 90, 30, 81, 1, 80, 19, 25, 76, 50, 77, 13, 65, 98, 26, 96, 24, 6, 87, 93, 55, 34, 78, 91, 69, 46, 49, 8, 36, 75, 85, 89, 42, 16, 84, 58, 11, 54, 67, 64, 68, 10, 66, 21, 57, 60, 28, 51, 3, 2, 53, 94, 17, 38, 45, 44, 95]


100%|██████████| 10000/10000 [01:23<00:00, 119.34it/s]
ic| f"Running {fname} -> N={n}, pop={pop}, steps={steps}, mut={mut}": 'Running problem_g_20.npy -> N=20, pop=10, steps=10000, mut=0.2'


...


== Result ==
file: problem_r2_50.npy
bestDistance: -1987.444416354721
bestSolution: [16, 30, 10, 13, 48, 35, 40, 44, 8, 27, 20, 34, 33, 47, 37, 45, 22, 4, 32, 36, 23, 26, 19, 18, 25, 5, 15, 29, 42, 11, 7, 28, 38, 9, 3, 24, 31, 14, 41, 1, 0, 12, 49, 17, 2, 46, 43, 39, 6, 21]


100%|██████████| 10000/10000 [00:43<00:00, 230.11it/s]
ic| f"Running {fname} -> N={n}, pop={pop}, steps={steps}, mut={mut}": 'Running test_problem.npy -> N=20, pop=10, steps=10000, mut=0.2'



== Result ==
file: problem_r2_500.npy
bestDistance: -14508.130399723106
bestSolution: [70, 19, 485, 277, 333, 192, 25, 252, 234, 169, 54, 7, 477, 464, 281, 340, 389, 173, 202, 398, 258, 138, 462, 441, 127, 431, 186, 272, 35, 165, 80, 314, 335, 488, 271, 39, 47, 44, 164, 144, 289, 347, 38, 226, 326, 265, 334, 85, 176, 489, 323, 151, 439, 261, 219, 136, 66, 183, 125, 318, 88, 16, 209, 191, 290, 114, 48, 235, 99, 433, 196, 363, 453, 376, 406, 172, 368, 89, 193, 416, 180, 242, 200, 153, 317, 160, 470, 167, 351, 499, 217, 345, 13, 305, 383, 299, 452, 213, 185, 446, 355, 97, 2, 381, 375, 474, 229, 379, 476, 218, 377, 490, 455, 350, 288, 492, 189, 21, 177, 401, 71, 338, 319, 132, 264, 469, 95, 417, 5, 284, 14, 98, 395, 459, 117, 223, 391, 59, 478, 78, 313, 491, 435, 65, 411, 26, 199, 287, 276, 131, 285, 367, 133, 315, 112, 3, 327, 128, 248, 360, 11, 109, 348, 447, 157, 60, 107, 79, 324, 178, 106, 224, 293, 75, 449, 442, 174, 17, 468, 415, 20, 472, 292, 241, 370, 460, 471, 484, 203, 437, 443,

100%|██████████| 10000/10000 [00:01<00:00, 6151.72it/s]


== Result ==
file: test_problem.npy
bestDistance: 2577.03
bestSolution: [16, 18, 19, 5, 3, 1, 11, 13, 10, 14, 6, 17, 7, 0, 2, 4, 9, 12, 15, 8]

=== SUMMARY (FULL) ===
file: problem_g_10.npy
bestDistance: 1243.5632731014168
bestSolution: [1, 6, 4, 5, 9, 7, 0, 8, 3, 2]
-
file: problem_g_100.npy
bestDistance: 9394.191746759203
bestSolution: [73, 70, 56, 18, 79, 35, 62, 23, 61, 59, 39, 82, 29, 9, 15, 83, 52, 32, 97, 72, 37, 33, 7, 27, 99, 71, 63, 47, 74, 20, 14, 41, 0, 48, 31, 12, 40, 43, 88, 92, 86, 4, 5, 22, 90, 30, 81, 1, 80, 19, 25, 76, 50, 77, 13, 65, 98, 26, 96, 24, 6, 87, 93, 55, 34, 78, 91, 69, 46, 49, 8, 36, 75, 85, 89, 42, 16, 84, 58, 11, 54, 67, 64, 68, 10, 66, 21, 57, 60, 28, 51, 3, 2, 53, 94, 17, 38, 45, 44, 95]
-
file: problem_g_1000.npy
bestDistance: 166609.60763394146
bestSolution: [532, 26, 436, 548, 618, 420, 576, 319, 636, 906, 276, 722, 292, 951, 195, 798, 361, 48, 78, 351, 559, 868, 641, 264, 97, 21, 613, 939, 536, 106, 259, 20, 328, 17, 115, 625, 176, 819, 104, 672, 